#### Milestone II
#### Michael Perrine
#### DSC 540
#### Professor Williams


# <h1><center>Market Analysis</center></h1>


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
cea = pd.read_csv(r"coincident economic activity index.csv")
cea = pd.DataFrame(cea)
cea = cea.iloc[::-1].reset_index(drop=True)
cea.dtypes

observation_date     object
USPHCI              float64
dtype: object

In [46]:
cli = pd.read_csv(r"composite leading indicators.csv")
cli = pd.DataFrame(cli)
cli = cli.iloc[::-1].reset_index(drop=True)
cli.head()


,observation_date,USALORSGPNOSTSAM
0,2023-11-01,100.4912
1,2023-10-01,100.4033
2,2023-09-01,100.3023
3,2023-08-01,100.1888
4,2023-07-01,100.0677


In [54]:
cpi = pd.read_csv(r"monthly cpi.csv")
cpi = pd.DataFrame(cpi)
cpi = cpi.iloc[::-1].reset_index(drop=True)
cpi.head()

,observation_date,CPIAUCSL
0,3/1/2025,319.615
1,2/1/2025,319.775
2,1/1/2025,319.086
3,12/1/2024,317.603
4,11/1/2024,316.449


In [55]:
mli = pd.read_csv(r"monthly leading indicators.csv")
mli = pd.DataFrame(mli)
mli = mli.iloc[::-1].reset_index(drop=True)
mli.head()

,observation_date,USSLIND
0,2/1/2020,1.72
1,1/1/2020,1.57
2,12/1/2019,1.48
3,11/1/2019,1.38
4,10/1/2019,1.41


In [ ]:
t_bill = pd.read_csv(r"one year t-bill.csv")
t_bill = pd.DataFrame(t_bill)
t_bill = t_bill.iloc[::-1].reset_index(drop=True)
t_bill.head()

In [ ]:
combo = cea.merge(cli, how = "outer")
combo

In [ ]:
combo = combo.merge(mli, how="outer")
combo

In [44]:
combo = combo.merge(t_bill, how="outer")
combo

,observation_date,USPHCI,CPIAUCSL,USALORSGPNOSTSAM,USSLIND,TB1YR
0,1/1/1947,NaN,21.480,NaN,NaN,NaN
1,1/1/1948,NaN,23.680,NaN,NaN,NaN
2,1/1/1949,NaN,24.010,NaN,NaN,NaN
3,1/1/1950,NaN,23.510,NaN,NaN,NaN
4,1/1/1951,NaN,25.380,NaN,NaN,NaN
...,...,...,...,...,...,...
1856,9/1/2020,122.45,259.997,NaN,NaN,0.13
1857,9/1/2021,130.21,273.942,NaN,NaN,0.07
1858,9/1/2022,136.71,296.421,NaN,NaN,3.73
1859,9/1/2023,140.69,307.374,NaN,NaN,5.15


In [49]:
combo = combo.set_axis(["date", "cea", "cpi", "cli", "mli", "t_bill"], axis=1)
combo

,date,cea,cpi,cli,mli,t_bill
0,1/1/1947,NaN,21.480,NaN,NaN,NaN
1,1/1/1948,NaN,23.680,NaN,NaN,NaN
2,1/1/1949,NaN,24.010,NaN,NaN,NaN
3,1/1/1950,NaN,23.510,NaN,NaN,NaN
4,1/1/1951,NaN,25.380,NaN,NaN,NaN
...,...,...,...,...,...,...
1856,9/1/2020,122.45,259.997,NaN,NaN,0.13
1857,9/1/2021,130.21,273.942,NaN,NaN,0.07
1858,9/1/2022,136.71,296.421,NaN,NaN,3.73
1859,9/1/2023,140.69,307.374,NaN,NaN,5.15


In [53]:
combo_1 = combo.iloc[::-1].reset_index(drop=True)
combo_1


,date,cea,cpi,cli,mli,t_bill
0,9/1/2024,144.20,314.851,NaN,NaN,3.87
1,9/1/2023,140.69,307.374,NaN,NaN,5.15
2,9/1/2022,136.71,296.421,NaN,NaN,3.73
3,9/1/2021,130.21,273.942,NaN,NaN,0.07
4,9/1/2020,122.45,259.997,NaN,NaN,0.13
...,...,...,...,...,...,...
1856,1/1/1951,NaN,25.380,NaN,NaN,NaN
1857,1/1/1950,NaN,23.510,NaN,NaN,NaN
1858,1/1/1949,NaN,24.010,NaN,NaN,NaN
1859,1/1/1948,NaN,23.680,NaN,NaN,NaN
